<a href="https://colab.research.google.com/github/victoralcantara75/erythrocytes-classification/blob/feature-extraction/TCC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#math
import numpy as np
import sklearn
import skimage
import sklearn.model_selection

#ts and keras
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator

#dimensionality
from sklearn.decomposition import PCA

#classificators
from sklearn.svm import SVC

#visualization
import matplotlib.pyplot as plt

#utils
import os
import imageio

In [2]:
#dataset
!git clone https://github.com/victoralcantara75/train-test-erythrocytes.git

Cloning into 'train-test-erythrocytes'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (766/766), done.
remote: Compressing objects: 100% (562/562), done.
remote: Total 766 (delta 203), reused 766 (delta 203), pack-reused 0
Receiving objects: 100% (766/766), 731.72 KiB | 756.00 KiB/s, done.
Resolving deltas: 100% (203/203), done.


In [3]:
batch = 16
epochs = 30
opt = 'adam'
lr = 0.001

In [4]:
def loadDir(round):
  train_dir = './train-test-erythrocytes/dataset/5-fold/round_'+str(round)+'/train'
  test_dir = './train-test-erythrocytes/dataset/5-fold/round_'+str(round)+'/test'
  return train_dir, test_dir

In [5]:
def createModel():

  base_model = ResNet50(weights='imagenet', include_top=False,
                                           input_shape=(90, 90, 3))
  # x = base_model.output
  # x = keras.layers.GlobalAveragePooling2D()(x)

  # x = keras.layers.Dense(1024, activation='relu')(x)
  # predictions = keras.layers.Dense(3, activation='softmax')(x)

  # model = keras.models.Model(inputs=base_model.input, outputs=predictions)
  # model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return base_model

In [6]:
def loadGenerator(train_dir, test_dir):

  train_datagen = ImageDataGenerator(
                          rescale=1. / 255,
                          shear_range=0.2,
                          zoom_range=0.2,
                          vertical_flip=True,
                          horizontal_flip=True)
  test_datagen = ImageDataGenerator(rescale=1. / 255)

  train_generator = train_datagen.flow_from_directory(train_dir, batch_size = batch)
  test_generator = test_datagen.flow_from_directory(test_dir, batch_size = batch)
  return train_generator, test_generator

In [17]:
def extract_features(base_model, train_imgs, sample_count):
  
  # features = np.zeros(shape=(sample_count, 8, 8, 2048))  # Must be equal to the output of the convolutional base
  feaitures = []
  labels = np.zeros(shape=(sample_count,3))

  i = 0
  for inputs_batch, labels_batch in train_imgs:
      features_batch = base_model.predict(inputs_batch)
      features[i * batch: (i + 1) * batch] = features_batch
      labels[i * batch: (i + 1) * batch] = labels_batch
      i += 1
      if i * batch >= sample_count:
          break
  return features, labels

In [19]:
accs = []

for i in range (1, 2):
  # print("ROUND ", i)
  train_dir, test_dir = loadDir(i)
  base_model = createModel()

  train_imgs, test_imgs = loadGenerator(train_dir, test_dir)
  train_count = len(train_imgs)
  
  features, labels = extract_features(base_model, train_imgs, train_count)

  print(features, labels)
  pca = PCA(n_components = 2)
  pca.fit(features)
  reduc_features = pca.transform(features)

  clf = SVC()
  print(reduc_features.ndim, labels.ndim)
  # clf.fit(features, labels)

  # clf.predict(test_imgs)

#   model.fit(train_imgs, epochs=epochs)
#   result = model.evaluate(train_imgs)

#   accs.append(result[1])

# print("Result: ", np.mean(accs))

Found 500 images belonging to 3 classes.
Found 126 images belonging to 3 classes.
[[[[0.         0.         0.         ... 2.04717684 0.
    0.        ]
   [0.         0.         0.         ... 1.60629201 0.
    0.        ]
   [0.         0.         0.         ... 1.35295498 0.
    0.        ]
   ...
   [0.         0.         0.01140189 ... 2.18625641 0.
    0.        ]
   [0.         0.         0.         ... 1.67218518 0.
    0.        ]
   [0.         0.         0.         ... 3.33193064 0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 2.20620704 0.
    0.        ]
   [0.         0.         0.         ... 0.72608066 0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.99544275 0.
    0.        ]
   [0.         0.         0.         ... 2.01997066 0.
    0.        ]
   [0.         0.         0.         ... 1.02977741 0.
    0.        ]]

  [[0.         0.         0.         ... 0.     

ValueError: ignored